In [1]:
import os
os.chdir("../")

In [159]:
import glob
import pandas as pd
import time

In [374]:
resultDir = 'results'
problem = 'facilities' # choices=['setcover', 'cauctions', 'facilities', 'indset']
sampling_Strategies = ['uniform5','depthK','depthK2'] # choices: uniform5, depthK, depthK2, depthK3
seeds = [0,1,2,3,4]
eval_files = glob.glob(f'{resultDir}/{problem}_*.csv')
eval_file = eval_files[-1]

In [376]:
df = pd.read_csv(eval_file)
df = pd.concat([df[df['type']=='small'], df[df['type']=='medium']])
df = df.astype({'nlps': float, 'nnodes' : float})

# 计算 one-shifted geometric means（未完成）
use uniform5 as basis, devide each instance's result by the corresponding one solved by uniform5. Then compute one-shifted geometric mean grouped by ['type','sampling_strategy']

In [307]:
# process_time = time.process_time()
# perf_time = time.perf_counter()
df2 = df[df['sampling_strategy'] == 'uniform5']
for i in range(0, len(df)):
    ref_row = df2[
        (df2['policy'] == df.loc[i].policy) &
        (df2['seed'] == df.loc[i].seed) &
        (df2['type'] == df.loc[i].type) &
        (df2['instance'] == df.loc[i].instance)
    ]
    df.at[i,'stime'] /= ref_row.stime
    df.at[i,'nnodes'] = df.at[i,'nnodes'].astype(float) / ref_row.nnodes.astype(float)
    df.at[i,'nlps'] = df.at[i,'nlps'].astype(float) / ref_row.nlps.astype(float)
    df.at[i,'walltime'] = df.at[i,'walltime'].astype(float) / ref_row.walltime.astype(float)
    df.at[i,'proctime'] = df.at[i,'proctime'].astype(float) / ref_row.proctime.astype(float)
 

In [253]:
import numpy as np

In [273]:
df.loc[(df['sampling_strategy'] == 'depthK') & (df['type']=='small'),'stime'].mean()

1.03691385447494

In [308]:
df.groupby(['type','sampling_strategy']).mean()

seed    nnodes      nlps     stime       gap  \
type   sampling_strategy                                                 
medium depthK                2  1.025238  1.023790  1.018855  0.000000   
       depthK2               2  1.062313  1.039587  1.028057  0.000000   
       uniform5              2  1.000000  1.000000  1.000000  0.000000   
small  depthK                2  1.007539  0.995745  0.988374  0.000131   
       depthK2               2  1.015996  1.007330  1.002060  0.000174   
       uniform5              2  1.000000  1.000000  1.000000  0.000191   

                          ndomchgs  ncutoffs  walltime  proctime  
type   sampling_strategy                                          
medium depthK                    0         0  1.018507  1.018637  
       depthK2                   0         0  1.028073  1.028072  
       uniform5                  0         0  1.000000  1.000000  
small  depthK                    0         0  0.988560  0.988305  
       depthK2                   0         0  1.002174  1.002097  
       uniform5                  0         0  1.000000  1.000000

In [288]:
from scipy import stats

In [290]:
stats.mstats.gmean(df.loc[(df['sampling_strategy'] == 'depthK') & (df['type']=='small'),'stime'] + 1) - 1

1.036213421932457

# 计算 standard deviations
Average per-instance standard deviation (percentage)

In [309]:
std_df = df.groupby(['type','sampling_strategy','instance']).std() / df.groupby(['type','sampling_strategy','instance']).mean()

In [310]:
std_df.groupby(['type','sampling_strategy']).mean()

seed    nnodes      nlps     stime       gap  \
type   sampling_strategy                                                     
medium depthK             0.790569  0.176241  0.158089  0.143044       NaN   
       depthK2            0.790569  0.183424  0.162547  0.162908       NaN   
       uniform5           0.790569  0.000000  0.000000  0.000000       NaN   
small  depthK             0.790569  0.128215  0.096285  0.081853  1.391291   
       depthK2            0.790569  0.140687  0.107897  0.097581  0.949817   
       uniform5           0.790569  0.000000  0.000000  0.000000  0.913410   

                          ndomchgs  ncutoffs  walltime  proctime  
type   sampling_strategy                                          
medium depthK                  NaN       NaN  0.142510  0.142588  
       depthK2                 NaN       NaN  0.162135  0.162145  
       uniform5                NaN       NaN  0.000000  0.000000  
small  depthK                  NaN       NaN  0.080604  0.080630  
       depthK2                 NaN       NaN  0.096427  0.096721  
       uniform5                NaN       NaN  0.000000  0.000000

# 计算 Wins

In [377]:
df_uniform5 = df[df['sampling_strategy']=='uniform5']
df_depthK = df[df['sampling_strategy']=='depthK']
df_depthK2 = df[df['sampling_strategy']=='depthK2']
df['Wins'] = 0

In [379]:
for i in range(0,len(df_uniform5)):
    uniform5_row = df_uniform5.iloc[i]
    depthK_row = df[
        (df['sampling_strategy'] == 'depthK') &
        (df['policy'] == uniform5_row.policy) &
        (df['seed'] == uniform5_row.seed) &
        (df['type'] == uniform5_row.type) &
        (df['instance'] == uniform5_row.instance)
    ]
    depthK2_row = df.loc[
        (df['sampling_strategy'] == 'depthK2') &
        (df['policy'] == uniform5_row.policy) &
        (df['seed'] == uniform5_row.seed) &
        (df['type'] == uniform5_row.type) &
        (df['instance'] == uniform5_row.instance)
    ]
    winner = np.argmin([uniform5_row.stime, depthK_row.stime.iloc[0], depthK2_row.stime.iloc[0]])
    winner = ['uniform5', 'depthK', 'depthK2'][winner]
    df.loc[ (df['sampling_strategy'] == winner) &
            (df['policy'] == uniform5_row.policy) &
            (df['seed'] == uniform5_row.seed) &
            (df['type'] == uniform5_row.type) &
            (df['instance'] == uniform5_row.instance), 'Wins'] += 1

In [380]:
df.groupby(['type', 'sampling_strategy']).sum()

seed    nnodes      nlps     stime       gap  \
type   sampling_strategy                                                 
medium depthK              200   64247.0   84807.0  36220.05  0.000000   
       depthK2             200   63639.0   83625.0  35724.68  0.000000   
       uniform5            200   64226.0   84277.0  36416.79  0.000000   
small  depthK              200  126692.0  152892.0  25820.21  0.013059   
       depthK2             200  124882.0  153451.0  26464.66  0.017420   
       uniform5            200  118822.0  145599.0  26367.50  0.019083   

                          ndomchgs  ncutoffs      walltime      proctime  Wins  
type   sampling_strategy                                                        
medium depthK                    0         0  36708.903560  36727.453125    25  
       depthK2                   0         0  36222.645429  36231.656250    42  
       uniform5                  0         0  36908.882507  36924.421875    33  
small  depthK                    0         0  26368.496764  26428.218750    39  
       depthK2                   0         0  27020.186683  27078.796875    36  
       uniform5                  0         0  26912.304422  26978.437500    25